In [32]:
import statsmodels.stats.api as sms
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import vstack, hstack, csc_matrix
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit,\
                                    cross_val_score, cross_validate, GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from collections import defaultdict
import time
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce

In [33]:
# сайты, время
times = ['time%d' % i for i in range(1, 11)]
sites = ['site%d' % i for i in range(1, 11)]

# трейн
df_tr = pd.read_csv('train_sessions.csv', parse_dates = times).set_index('session_id').sort_values('time1')

In [121]:
def get_AB_required_n(prop1, prop2, alpha, power=.8):
    'считает необходимый размер выборки дляпроведения АБ теста'
    effect_size = sms.proportion_effectsize(prop1, prop2)
    return np.int32(np.around(sms.NormalIndPower().solve_power(effect_size, power=power,alpha=alpha, ratio=1)))

In [122]:
sample_size = get_AB_required_n(prop1 = .14, prop2 = .15, alpha = .05, power = .8)

In [123]:
sample_size

19456

In [71]:
alpha = .05

In [126]:
results_bt, results_ttest = [], []
for _ in range(1, 51):

    sample1 = np.log1p(df_tr[sites].sample(sample_size, random_state = _))
    sample2 = np.log1p(df_tr[sites].drop(sample1.index).sample(sample_size, random_state = _))

    sample1 = pd.Series(sample1.values.flatten()).dropna()
    sample2 = pd.Series(sample2.values.flatten()).dropna()
    
    if len(means_A) == len(means_B):
        if stats.ttest_ind(sample1, sample2, equal_var = False)[1] > alpha:
            results_ttest.append('различий нет')
        else:
            results_ttest.append('различия есть')            
    else:
        if stats.ttest_ind(sample1, sample2, equal_var = True)[1] > alpha:
            results_ttest.append('различий нет')
        else:
            results_ttest.append('различия есть')     
            
        
    
    np.random.seed(_)
    means_A = np.mean(np.random.choice(sample1, (1000, 100)), 1)
    means_B = np.mean(np.random.choice(sample2, (1000, 100)), 1)

    if len(means_A) == len(means_B):
        if stats.ttest_ind(means_A, means_B, equal_var = True)[1] > alpha:
            results_bt.append('различий нет')
        else:
            results_bt.append('различия есть')
            
    else:
        if stats.ttest_ind(means_A, means_B)[1] > alpha:
            results_bt.append('различий нет')
        else:
            results_bt.append('различия есть')
            
        


In [112]:
pd.Series(results_bt).value_counts().idxmax()

'различий нет'

In [113]:
pd.Series(results_ttest).value_counts().idxmax()

'различий нет'